## Step 1: Install Dependencies


In [ ]:
!pip3 install ir-datasets 'python-terrier==0.10.0' 'git+https://github.com/tira-io/tira.git@pyterrier-artifacts#egg=tira&subdirectory=python-client' 'git+https://github.com/webis-de/auto-ir-metadata@dev' 'git+https://github.com/mam10eks/autoqrels.git' 'git+https://github.com/OpenWebSearch/wows-code.git#egg=wows-eval&subdirectory=ecir25/wows-eval' --break-system-packages

In [2]:
from wows_eval import evaluate as wows_evaluate

from autoqrels.oneshot import DuoPrompt
from auto_ir_metadata import Environment
import pandas as pd
from tira.rest_api_client import Client

tira = Client()
pd.set_option('display.max_colwidth', None)

WOWS_DATASET_ID = 'wows-eval/pairwise-smoke-test-20250210-training'


In [3]:
BACKBONE_MODEL = "flan-t5-small"

PROMPT = """Determine if passage B is as relevant as passage A
for the given query.
'Passage A: "...{{ rel_doc_text | replace("\\"", "\'") }}..."
'Passage B: "...{{ unk_doc_text | replace("\\"", "\'") }}..."
'Query: "{{ query_text }}" '
"Is passage B as relevant as passage A? </s>"""

In [4]:
flan_t5_small_assessor = DuoPrompt(
    backbone=f'google/{BACKBONE_MODEL}',
    prompt=PROMPT,
    dataset=None
)

In [5]:
input_data = tira.pd.inputs(WOWS_DATASET_ID)
input_data.head(3)

,id,query,relevant,unknown
0,3d080873-98a1-4388-af86-fe2c8b47ebca,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Randy Newman (album) Randy Newman is the debut recording by Randy Newman, released in 1968. Unlike his later albums which featured Newman and his piano backed by guitar, bass guitar and drums, Randy Newman was highly orchestral and aimed to blend the orchestra with Newman's voice and piano."
1,468a9e92-467f-47c9-810b-fe6fa9dca634,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"One of Monk's most important contributions to jazz was his use of space and simplicity in his performances and in his compositions. Monk's unconventional use of harmony and rhythm has had a lasting influence on jazz as well. Finally, Monk's compositions are very well-known, both by performers and by listeners."
2,846a69d0-0c0e-4d86-baf2-c3e8d31fdc86,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Singing elegant, melancholic songs in a glamorously tattered voice, Leonard Cohen emerged from Montreal in the 1960s, an artist well into his thirties before he even made his first album. After a few records, he was royalty, on equal footing with Joni Mitchell, Randy Newman, and other top-notch singer-songwriters."


In [7]:
with Environment().measure() as tracked_experiment:
    predictions = flan_t5_small_assessor.predict(input_data)

print(wows_evaluate(
    predictions,
    WOWS_DATASET_ID,
    upload=True,
    system_name=f'auto-qrels-pairwise-{BACKBONE_MODEL}',
    system_description="We use autoqrels [1] with a custom in-context learning prompt for pairwise relevance judgments.\n\n[1] - https://github.com/seanmacavaney/autoqrels",
    environment=tracked_experiment
))

predictions.head(3)

100%|██████████| 6/6 [00:03<00:00,  1.70it/s]


Run uploaded to TIRA. Claim ownership via: https://www.tira.io/claim-submission/0ba3edaa-bc58-42ac-a592-6d3dd026691e
{'tau_ap': -0.3038888888888889, 'kendall': -0.1523809523809524, 'spearman': -0.1785714285714286, 'pearson': -0.1785714285714286}


,id,query,relevant,unknown,probability_relevant
0,3d080873-98a1-4388-af86-fe2c8b47ebca,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Randy Newman (album) Randy Newman is the debut recording by Randy Newman, released in 1968. Unlike his later albums which featured Newman and his piano backed by guitar, bass guitar and drums, Randy Newman was highly orchestral and aimed to blend the orchestra with Newman's voice and piano.",0.616158
1,468a9e92-467f-47c9-810b-fe6fa9dca634,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"One of Monk's most important contributions to jazz was his use of space and simplicity in his performances and in his compositions. Monk's unconventional use of harmony and rhythm has had a lasting influence on jazz as well. Finally, Monk's compositions are very well-known, both by performers and by listeners.",0.713638
2,846a69d0-0c0e-4d86-baf2-c3e8d31fdc86,who sings monk theme song,exists and is an alternate of . The Monk theme song is It's a Jungle Out There by Randy Newman. The Monk theme song is It's a Jungle Out There by Randy Newman.,"Singing elegant, melancholic songs in a glamorously tattered voice, Leonard Cohen emerged from Montreal in the 1960s, an artist well into his thirties before he even made his first album. After a few records, he was royalty, on equal footing with Joni Mitchell, Randy Newman, and other top-notch singer-songwriters.",0.684874
